In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine


In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m'),("spark.sql.execution.arrow.enabled", "true")]) # ("spark.sql.execution.arrow.enabled", "true")


In [4]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)
dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist").option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
dataframe_mysql.registerTempTable('starlist')

Exception: Java gateway process exited before sending its port number

In [5]:
def innerJoin(tableName,provDir,totGross,diffSize=1,xlength=1015.,ylength=1085.):
    nsegs = 4
    
    temptableName = tableName[0:5]+tableName[-1]
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE, ELLIPTICITY FROM {} INNER JOIN starlist ON {}.X_IMAGE BETWEEN starlist.X_POS_MIN AND starlist.X_POS_MAX AND {}.Y_IMAGE BETWEEN starlist.Y_POS_MIN AND starlist.Y_POS_MAX".format(temptableName,temptableName,temptableName))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    #print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    
    variableList = [tableName]
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+15
        xmax = int(xmin+xsegment)+15
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+15
            ymax = int(ymin+ysegment)+15
            
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            
            number = df.count()
            #print('x {} y {} num {}'.format(xsegs,ysegs,number))
            variableList.append(number)
            # need to create some place to write to
    magRange = np.linspace(13,28,16)
    #magRange = [] # update with observed mag range from sextractor
    for mag in range(len(magRange)):
        if magRange[mag] != magRange[-1]:
            magMin = magRange[mag]
            magMax = magRange[mag+1]
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.MAG_BEST BETWEEN {1} AND {2}".format(temptableName+'match',magMin,magMax))
            number = df.count()
            #print('mag {} number {}'.format(mag,number))
            variableList.append(number)
            # record results somewhere
    
    
    
    astroprov.provcall([tableName,'starlist'],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableTuple = tuple(variableList)
    #print(variableTuple)
    return variableTuple

def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    
    #workAroundList = [16,32,64,128,256,512]
    workAroundList = [1,3,5,7,9,11]
    ident = workAroundList.index(valList[-1])
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test'+str(ident)+'.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_'+str(ident)+'.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    astroprov.provcall(valList,[confName],"makeConfig_Python2Python_SQ_tmpl.provn","makeConfig",provDir)
    return(confName,tableName,catname)



def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits'):
    
    astroprov.provcall([confName,'/home/mj1e16/iraf/editedImage5000.fits'],[tableName],"findObjects_Python2DaoStarfidner_SQ_tmpl.provn","findObjects",provDir)

    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    #print(temptableName)
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    #print('Gross ',totGross)
    sqlContext.cacheTable(temptableName)
    # count rows in table and add to some big table
    
    #df_spark.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    return totGross

def alltogethernow(valList,tableName,IMAGE,provDir):
    Names = makeConfig(valList,provDir,tableName=tableName)
    totGross = findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],provDir,totGross)
    return finalTuple

In [6]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]]

In [7]:
def evaluateImage(valList,simImage,provDir,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    
    finalTableTuples = []
    #p = ThreadPool(1)
    t0 = time.time()
    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                for x2 in range(len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    for x3 in range(len(valList[3])):
                        fullname = name+str(x3)
                        fullValList = [valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]]
                        print(fullname)
                        try:
                            finalTableTuples.append(alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,provDir=provDir))
                        except RuntimeError:
                            with open('redos','a') as f:
                                bigString = simImage+','+str(fullname)+','+str(fullValList)
                                f.write(bigString)
                            print('Redo with valList = ',fullValList)
                            pass

    rdd = sc.parallelize(finalTableTuples)
    #print(finalTableTuples)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xy_0=int(x[1]),xy_1=int(x[2]),xy_2=int(x[3]),xy_3=int(x[4]),xy_4=int(x[5]),xy_5=int(x[6]),xy_6=int(x[7]),xy_7=int(x[8]),xy_8=int(x[9]),xy_9=int(x[10]),xy_10=int(x[11]),xy_11=int(x[12]),xy_12=int(x[13]),xy_13=int(x[14]),xy_14=int(x[15]),xy_15=int(x[16]),mag_0=int(x[17]),mag_1=int(x[18]),mag_2=int(x[19]),mag_3=int(x[20]),mag_4=int(x[21]),mag_5=int(x[22]),mag_6=int(x[23]),mag_7=int(x[24]),mag_8=int(x[25]),mag_9=int(x[26]),mag_10=int(x[27]),mag_11=int(x[28]),mag_12=int(x[29]),mag_13=int(x[30]),mag_14=int(x[31]),totNum=int(x[32]),totGross=int(x[33])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResults_44_1',user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    print(time.time()-t0)

In [8]:
sqlContext.cacheTable('starlist')

In [9]:
#sqlContext.sql('DROP TABLE IF EXISTS finalResults_44_1')

DataFrame[]

In [10]:
### Make function to replace starlist
evaluateImage(valList,'/home/mj1e16/Simages/diff_44_1_alt.fits','/home/mj1e16/keplerPhotometry/provDump/44/selectiveMedian/',smallName='Poshak_44_1')

Poshak_44_1_0_0_0_0
Poshak_44_1_0_0_0_1
Poshak_44_1_0_0_0_2
Poshak_44_1_0_0_0_3
Poshak_44_1_0_0_0_4
Poshak_44_1_0_0_0_5
Poshak_44_1_0_0_1_0
Poshak_44_1_0_0_1_1
Poshak_44_1_0_0_1_2
Poshak_44_1_0_0_1_3
Poshak_44_1_0_0_1_4
Poshak_44_1_0_0_1_5
Poshak_44_1_0_0_2_0
Poshak_44_1_0_0_2_1
Poshak_44_1_0_0_2_2
Poshak_44_1_0_0_2_3
Poshak_44_1_0_0_2_4
Poshak_44_1_0_0_2_5
Poshak_44_1_0_0_3_0
Poshak_44_1_0_0_3_1
Poshak_44_1_0_0_3_2
Poshak_44_1_0_0_3_3
Poshak_44_1_0_0_3_4
Poshak_44_1_0_0_3_5
Poshak_44_1_0_0_4_0
Poshak_44_1_0_0_4_1
Poshak_44_1_0_0_4_2
Poshak_44_1_0_0_4_3
Poshak_44_1_0_0_4_4
Poshak_44_1_0_0_4_5
Poshak_44_1_0_0_5_0
Poshak_44_1_0_0_5_1
Poshak_44_1_0_0_5_2
Poshak_44_1_0_0_5_3
Poshak_44_1_0_0_5_4
Poshak_44_1_0_0_5_5
Poshak_44_1_0_0_6_0
Poshak_44_1_0_0_6_1
Poshak_44_1_0_0_6_2
Poshak_44_1_0_0_6_3
Poshak_44_1_0_0_6_4
Poshak_44_1_0_0_6_5
Poshak_44_1_0_0_7_0
Poshak_44_1_0_0_7_1
Poshak_44_1_0_0_7_2
Poshak_44_1_0_0_7_3
Poshak_44_1_0_0_7_4
Poshak_44_1_0_0_7_5
Poshak_44_1_0_0_8_0
Poshak_44_1_0_0_8_1


Poshak_44_1_0_4_3_1
Poshak_44_1_0_4_3_2
Poshak_44_1_0_4_3_3
Poshak_44_1_0_4_3_4
Poshak_44_1_0_4_3_5
Poshak_44_1_0_4_4_0
Poshak_44_1_0_4_4_1
Poshak_44_1_0_4_4_2
Poshak_44_1_0_4_4_3
Poshak_44_1_0_4_4_4
Poshak_44_1_0_4_4_5
Poshak_44_1_0_4_5_0
Poshak_44_1_0_4_5_1
Poshak_44_1_0_4_5_2
Poshak_44_1_0_4_5_3
Poshak_44_1_0_4_5_4
Poshak_44_1_0_4_5_5
Poshak_44_1_0_4_6_0
Poshak_44_1_0_4_6_1
Poshak_44_1_0_4_6_2
Poshak_44_1_0_4_6_3
Poshak_44_1_0_4_6_4
Poshak_44_1_0_4_6_5
Poshak_44_1_0_4_7_0
Poshak_44_1_0_4_7_1
Poshak_44_1_0_4_7_2
Poshak_44_1_0_4_7_3
Poshak_44_1_0_4_7_4
Poshak_44_1_0_4_7_5
Poshak_44_1_0_4_8_0
Poshak_44_1_0_4_8_1
Poshak_44_1_0_4_8_2
Poshak_44_1_0_4_8_3
Poshak_44_1_0_4_8_4
Poshak_44_1_0_4_8_5
Poshak_44_1_0_4_9_0
Poshak_44_1_0_4_9_1
Poshak_44_1_0_4_9_2
Poshak_44_1_0_4_9_3
Poshak_44_1_0_4_9_4
Poshak_44_1_0_4_9_5
Poshak_44_1_0_4_10_0
Poshak_44_1_0_4_10_1
Poshak_44_1_0_4_10_2
Poshak_44_1_0_4_10_3
Poshak_44_1_0_4_10_4
Poshak_44_1_0_4_10_5
Poshak_44_1_0_4_11_0
Poshak_44_1_0_4_11_1
Poshak_44_1_

Poshak_44_1_0_8_6_2
Poshak_44_1_0_8_6_3
Poshak_44_1_0_8_6_4
Poshak_44_1_0_8_6_5
Poshak_44_1_0_8_7_0
Poshak_44_1_0_8_7_1
Poshak_44_1_0_8_7_2
Poshak_44_1_0_8_7_3
Poshak_44_1_0_8_7_4
Poshak_44_1_0_8_7_5
Poshak_44_1_0_8_8_0
Poshak_44_1_0_8_8_1
Poshak_44_1_0_8_8_2
Poshak_44_1_0_8_8_3
Poshak_44_1_0_8_8_4
Poshak_44_1_0_8_8_5
Poshak_44_1_0_8_9_0
Poshak_44_1_0_8_9_1
Poshak_44_1_0_8_9_2
Poshak_44_1_0_8_9_3
Poshak_44_1_0_8_9_4
Poshak_44_1_0_8_9_5
Poshak_44_1_0_8_10_0
Poshak_44_1_0_8_10_1
Poshak_44_1_0_8_10_2
Poshak_44_1_0_8_10_3
Poshak_44_1_0_8_10_4
Poshak_44_1_0_8_10_5
Poshak_44_1_0_8_11_0
Poshak_44_1_0_8_11_1
Poshak_44_1_0_8_11_2
Poshak_44_1_0_8_11_3
Poshak_44_1_0_8_11_4
Poshak_44_1_0_8_11_5
Poshak_44_1_0_8_12_0
Poshak_44_1_0_8_12_1
Poshak_44_1_0_8_12_2
Poshak_44_1_0_8_12_3
Poshak_44_1_0_8_12_4
Poshak_44_1_0_8_12_5
Poshak_44_1_0_8_13_0
Poshak_44_1_0_8_13_1
Poshak_44_1_0_8_13_2
Poshak_44_1_0_8_13_3
Poshak_44_1_0_8_13_4
Poshak_44_1_0_8_13_5
Poshak_44_1_0_8_14_0
Poshak_44_1_0_8_14_1
Poshak_44_1_0_

Poshak_44_1_1_3_9_3
Poshak_44_1_1_3_9_4
Poshak_44_1_1_3_9_5
Poshak_44_1_1_3_10_0
Poshak_44_1_1_3_10_1
Poshak_44_1_1_3_10_2
Poshak_44_1_1_3_10_3
Poshak_44_1_1_3_10_4
Poshak_44_1_1_3_10_5
Poshak_44_1_1_3_11_0
Poshak_44_1_1_3_11_1
Poshak_44_1_1_3_11_2
Poshak_44_1_1_3_11_3
Poshak_44_1_1_3_11_4
Poshak_44_1_1_3_11_5
Poshak_44_1_1_3_12_0
Poshak_44_1_1_3_12_1
Poshak_44_1_1_3_12_2
Poshak_44_1_1_3_12_3
Poshak_44_1_1_3_12_4
Poshak_44_1_1_3_12_5
Poshak_44_1_1_3_13_0
Poshak_44_1_1_3_13_1
Poshak_44_1_1_3_13_2
Poshak_44_1_1_3_13_3
Poshak_44_1_1_3_13_4
Poshak_44_1_1_3_13_5
Poshak_44_1_1_3_14_0
Poshak_44_1_1_3_14_1
Poshak_44_1_1_3_14_2
Poshak_44_1_1_3_14_3
Poshak_44_1_1_3_14_4
Poshak_44_1_1_3_14_5
Poshak_44_1_1_3_15_0
Poshak_44_1_1_3_15_1
Poshak_44_1_1_3_15_2
Poshak_44_1_1_3_15_3
Poshak_44_1_1_3_15_4
Poshak_44_1_1_3_15_5
Poshak_44_1_1_4_0_0
Poshak_44_1_1_4_0_1
Poshak_44_1_1_4_0_2
Poshak_44_1_1_4_0_3
Poshak_44_1_1_4_0_4
Poshak_44_1_1_4_0_5
Poshak_44_1_1_4_1_0
Poshak_44_1_1_4_1_1
Poshak_44_1_1_4_1_2
Posh

Poshak_44_1_1_7_12_3
Poshak_44_1_1_7_12_4
Poshak_44_1_1_7_12_5
Poshak_44_1_1_7_13_0
Poshak_44_1_1_7_13_1
Poshak_44_1_1_7_13_2
Poshak_44_1_1_7_13_3
Poshak_44_1_1_7_13_4
Poshak_44_1_1_7_13_5
Poshak_44_1_1_7_14_0
Poshak_44_1_1_7_14_1
Poshak_44_1_1_7_14_2
Poshak_44_1_1_7_14_3
Poshak_44_1_1_7_14_4
Poshak_44_1_1_7_14_5
Poshak_44_1_1_7_15_0
Poshak_44_1_1_7_15_1
Poshak_44_1_1_7_15_2
Poshak_44_1_1_7_15_3
Poshak_44_1_1_7_15_4
Poshak_44_1_1_7_15_5
Poshak_44_1_1_8_0_0
Poshak_44_1_1_8_0_1
Poshak_44_1_1_8_0_2
Poshak_44_1_1_8_0_3
Poshak_44_1_1_8_0_4
Poshak_44_1_1_8_0_5
Poshak_44_1_1_8_1_0
Poshak_44_1_1_8_1_1
Poshak_44_1_1_8_1_2
Poshak_44_1_1_8_1_3
Poshak_44_1_1_8_1_4
Poshak_44_1_1_8_1_5
Poshak_44_1_1_8_2_0
Poshak_44_1_1_8_2_1
Poshak_44_1_1_8_2_2
Poshak_44_1_1_8_2_3
Poshak_44_1_1_8_2_4
Poshak_44_1_1_8_2_5
Poshak_44_1_1_8_3_0
Poshak_44_1_1_8_3_1
Poshak_44_1_1_8_3_2
Poshak_44_1_1_8_3_3
Poshak_44_1_1_8_3_4
Poshak_44_1_1_8_3_5
Poshak_44_1_1_8_4_0
Poshak_44_1_1_8_4_1
Poshak_44_1_1_8_4_2
Poshak_44_1_1_8_4_3

...

Poshak_44_1_7_8_4_4
Poshak_44_1_7_8_4_5
Poshak_44_1_7_8_5_0
Poshak_44_1_7_8_5_1
Poshak_44_1_7_8_5_2
Poshak_44_1_7_8_5_3
Poshak_44_1_7_8_5_4
Poshak_44_1_7_8_5_5
Poshak_44_1_7_8_6_0
Poshak_44_1_7_8_6_1
Poshak_44_1_7_8_6_2
Poshak_44_1_7_8_6_3
Poshak_44_1_7_8_6_4
Poshak_44_1_7_8_6_5
Poshak_44_1_7_8_7_0
Poshak_44_1_7_8_7_1
Poshak_44_1_7_8_7_2
Poshak_44_1_7_8_7_3
Poshak_44_1_7_8_7_4
Poshak_44_1_7_8_7_5
Poshak_44_1_7_8_8_0
Poshak_44_1_7_8_8_1
Poshak_44_1_7_8_8_2
Poshak_44_1_7_8_8_3
Poshak_44_1_7_8_8_4
Poshak_44_1_7_8_8_5
Poshak_44_1_7_8_9_0
Poshak_44_1_7_8_9_1
Poshak_44_1_7_8_9_2
Poshak_44_1_7_8_9_3
Poshak_44_1_7_8_9_4
Poshak_44_1_7_8_9_5
Poshak_44_1_7_8_10_0
Poshak_44_1_7_8_10_1
Poshak_44_1_7_8_10_2
Poshak_44_1_7_8_10_3
Poshak_44_1_7_8_10_4
Poshak_44_1_7_8_10_5
Poshak_44_1_7_8_11_0
Poshak_44_1_7_8_11_1
Poshak_44_1_7_8_11_2
Poshak_44_1_7_8_11_3
Poshak_44_1_7_8_11_4
Poshak_44_1_7_8_11_5
Poshak_44_1_7_8_12_0
Poshak_44_1_7_8_12_1
Poshak_44_1_7_8_12_2
Poshak_44_1_7_8_12_3
Poshak_44_1_7_8_12_4
Pos

Poshak_44_1_8_3_7_5
Poshak_44_1_8_3_8_0
Poshak_44_1_8_3_8_1
Poshak_44_1_8_3_8_2
Poshak_44_1_8_3_8_3
Poshak_44_1_8_3_8_4
Poshak_44_1_8_3_8_5
Poshak_44_1_8_3_9_0
Poshak_44_1_8_3_9_1
Poshak_44_1_8_3_9_2
Poshak_44_1_8_3_9_3
Poshak_44_1_8_3_9_4
Poshak_44_1_8_3_9_5
Poshak_44_1_8_3_10_0
Poshak_44_1_8_3_10_1
Poshak_44_1_8_3_10_2
Poshak_44_1_8_3_10_3
Poshak_44_1_8_3_10_4
Poshak_44_1_8_3_10_5
Poshak_44_1_8_3_11_0
Poshak_44_1_8_3_11_1
Poshak_44_1_8_3_11_2
Poshak_44_1_8_3_11_3
Poshak_44_1_8_3_11_4
Poshak_44_1_8_3_11_5
Poshak_44_1_8_3_12_0
Poshak_44_1_8_3_12_1
Poshak_44_1_8_3_12_2
Poshak_44_1_8_3_12_3
Poshak_44_1_8_3_12_4
Poshak_44_1_8_3_12_5
Poshak_44_1_8_3_13_0
Poshak_44_1_8_3_13_1
Poshak_44_1_8_3_13_2
Poshak_44_1_8_3_13_3
Poshak_44_1_8_3_13_4
Poshak_44_1_8_3_13_5
Poshak_44_1_8_3_14_0
Poshak_44_1_8_3_14_1
Poshak_44_1_8_3_14_2
Poshak_44_1_8_3_14_3
Poshak_44_1_8_3_14_4
Poshak_44_1_8_3_14_5
Poshak_44_1_8_3_15_0
Poshak_44_1_8_3_15_1
Poshak_44_1_8_3_15_2
Poshak_44_1_8_3_15_3
Poshak_44_1_8_3_15_4
Posha

Poshak_44_1_8_7_11_0
Poshak_44_1_8_7_11_1
Poshak_44_1_8_7_11_2
Poshak_44_1_8_7_11_3
Poshak_44_1_8_7_11_4
Poshak_44_1_8_7_11_5
Poshak_44_1_8_7_12_0
Poshak_44_1_8_7_12_1
Poshak_44_1_8_7_12_2
Poshak_44_1_8_7_12_3
Poshak_44_1_8_7_12_4
Poshak_44_1_8_7_12_5
Poshak_44_1_8_7_13_0
Poshak_44_1_8_7_13_1
Poshak_44_1_8_7_13_2
Poshak_44_1_8_7_13_3
Poshak_44_1_8_7_13_4
Poshak_44_1_8_7_13_5
Poshak_44_1_8_7_14_0
Poshak_44_1_8_7_14_1
Poshak_44_1_8_7_14_2
Poshak_44_1_8_7_14_3
Poshak_44_1_8_7_14_4
Poshak_44_1_8_7_14_5
Poshak_44_1_8_7_15_0
Poshak_44_1_8_7_15_1
Poshak_44_1_8_7_15_2
Poshak_44_1_8_7_15_3
Poshak_44_1_8_7_15_4
Poshak_44_1_8_7_15_5
Poshak_44_1_8_8_0_0
Poshak_44_1_8_8_0_1
Poshak_44_1_8_8_0_2
Poshak_44_1_8_8_0_3
Poshak_44_1_8_8_0_4
Poshak_44_1_8_8_0_5
Poshak_44_1_8_8_1_0
Poshak_44_1_8_8_1_1
Poshak_44_1_8_8_1_2
Poshak_44_1_8_8_1_3
Poshak_44_1_8_8_1_4
Poshak_44_1_8_8_1_5
Poshak_44_1_8_8_2_0
Poshak_44_1_8_8_2_1
Poshak_44_1_8_8_2_2
Poshak_44_1_8_8_2_3
Poshak_44_1_8_8_2_4
Poshak_44_1_8_8_2_5
Poshak_44_

Poshak_44_1_9_2_14_0
Poshak_44_1_9_2_14_1
Poshak_44_1_9_2_14_2
Poshak_44_1_9_2_14_3
Poshak_44_1_9_2_14_4
Poshak_44_1_9_2_14_5
Poshak_44_1_9_2_15_0
Poshak_44_1_9_2_15_1
Poshak_44_1_9_2_15_2
Poshak_44_1_9_2_15_3
Poshak_44_1_9_2_15_4
Poshak_44_1_9_2_15_5
Poshak_44_1_9_3_0_0
Poshak_44_1_9_3_0_1
Poshak_44_1_9_3_0_2
Poshak_44_1_9_3_0_3
Poshak_44_1_9_3_0_4
Poshak_44_1_9_3_0_5
Poshak_44_1_9_3_1_0
Poshak_44_1_9_3_1_1
Poshak_44_1_9_3_1_2
Poshak_44_1_9_3_1_3
Poshak_44_1_9_3_1_4
Poshak_44_1_9_3_1_5
Poshak_44_1_9_3_2_0
Poshak_44_1_9_3_2_1
Poshak_44_1_9_3_2_2
Poshak_44_1_9_3_2_3
Poshak_44_1_9_3_2_4
Poshak_44_1_9_3_2_5
Poshak_44_1_9_3_3_0
Poshak_44_1_9_3_3_1
Poshak_44_1_9_3_3_2
Poshak_44_1_9_3_3_3
Poshak_44_1_9_3_3_4
Poshak_44_1_9_3_3_5
Poshak_44_1_9_3_4_0
Poshak_44_1_9_3_4_1
Poshak_44_1_9_3_4_2
Poshak_44_1_9_3_4_3
Poshak_44_1_9_3_4_4
Poshak_44_1_9_3_4_5
Poshak_44_1_9_3_5_0
Poshak_44_1_9_3_5_1
Poshak_44_1_9_3_5_2
Poshak_44_1_9_3_5_3
Poshak_44_1_9_3_5_4
Poshak_44_1_9_3_5_5
Poshak_44_1_9_3_6_0
Poshak_4

Poshak_44_1_9_7_1_0
Poshak_44_1_9_7_1_1
Poshak_44_1_9_7_1_2
Poshak_44_1_9_7_1_3
Poshak_44_1_9_7_1_4
Poshak_44_1_9_7_1_5
Poshak_44_1_9_7_2_0
Poshak_44_1_9_7_2_1
Poshak_44_1_9_7_2_2
Poshak_44_1_9_7_2_3
Poshak_44_1_9_7_2_4
Poshak_44_1_9_7_2_5
Poshak_44_1_9_7_3_0
Poshak_44_1_9_7_3_1
Poshak_44_1_9_7_3_2
Poshak_44_1_9_7_3_3
Poshak_44_1_9_7_3_4
Poshak_44_1_9_7_3_5
Poshak_44_1_9_7_4_0
Poshak_44_1_9_7_4_1
Poshak_44_1_9_7_4_2
Poshak_44_1_9_7_4_3
Poshak_44_1_9_7_4_4
Poshak_44_1_9_7_4_5
Poshak_44_1_9_7_5_0
Poshak_44_1_9_7_5_1
Poshak_44_1_9_7_5_2
Poshak_44_1_9_7_5_3
Poshak_44_1_9_7_5_4
Poshak_44_1_9_7_5_5
Poshak_44_1_9_7_6_0
Poshak_44_1_9_7_6_1
Poshak_44_1_9_7_6_2
Poshak_44_1_9_7_6_3
Poshak_44_1_9_7_6_4
Poshak_44_1_9_7_6_5
Poshak_44_1_9_7_7_0
Poshak_44_1_9_7_7_1
Poshak_44_1_9_7_7_2
Poshak_44_1_9_7_7_3
Poshak_44_1_9_7_7_4
Poshak_44_1_9_7_7_5
Poshak_44_1_9_7_8_0
Poshak_44_1_9_7_8_1
Poshak_44_1_9_7_8_2
Poshak_44_1_9_7_8_3
Poshak_44_1_9_7_8_4
Poshak_44_1_9_7_8_5
Poshak_44_1_9_7_9_0
Poshak_44_1_9_7_9_1


In [14]:
tup = [(1,2,3),(4,5,6)]
rdd = sc.parallelize(tup)
kepler = rdd.map(lambda x: Row(one=int(x[0]),two=int(x[1]),three=int(x[2])))
schemaKepler = sqlContext.createDataFrame(kepler)
schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='testTab',user='mj1e16',password='[sqlT1G3R]').mode('append').save()


In [15]:
a = sqlContext.sql("show tables").show()
print(a)

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        | starlist|       true|
+--------+---------+-----------+

None


In [15]:
def test():
    row = [('tabName',1,1),('tabName2',1,2)]
    rdd = sc.parallelize(row)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xyRegion=int(x[1]),magRegion=int(x[2])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    df = schemaKepler.toPandas()
    print(df)

In [25]:
def testPartial(value):
    return (value) #,value)

In [26]:
p = ThreadPool(6)
values = [0,1,2,3,4,5]
answer = []
for x in range(5):
    answer.extend(p.map(testPartial,values))
print(answer)

[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]


In [26]:
import sys

In [27]:
sys.getsizeof(answer)

336

In [12]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
    maxVal = df2['MAG_BEST'].max()
    minVal = df2['MAG_BEST'].min()
#     df4 = df2.sort_values('MAG_BEST')
#     return df4
    print(minVal,maxVal)

In [32]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
#     maxVal = df2['MAG_BEST'].max()
#     minVal = df2['MAG_BEST'].min()
    df4 = df2.sort_values('MAG_BEST')
    return df4
#     print(minVal,maxVal)

In [15]:
for x in range(6):
    print('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))
    findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

/home/mj1e16/sextractor/sextractor-master/config/test0.cat
/home/mj1e16/sextractor/sextractor-master/config/test1.cat
/home/mj1e16/sextractor/sextractor-master/config/test2.cat
/home/mj1e16/sextractor/sextractor-master/config/test3.cat
/home/mj1e16/sextractor/sextractor-master/config/test4.cat
/home/mj1e16/sextractor/sextractor-master/config/test5.cat


In [33]:
x = 2
df = findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

In [35]:
df

,NUMBER,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY
3025,3026,13.0256,782.0090,185.0009,0.352
2869,2870,13.0305,1052.8906,184.9989,0.036
13573,13574,13.0327,1052.7808,946.0535,0.125
13344,13345,13.0328,239.7816,946.0737,0.031
6969,6970,13.0329,239.5148,437.8912,0.103
11193,11194,13.0329,239.6400,691.9606,0.054
13666,13667,13.0333,781.5560,946.0166,0.260
6968,6969,13.0335,781.4474,438.1546,0.157
7010,7011,13.0338,510.7693,437.9970,0.031
2859,2860,13.0340,239.8327,185.1199,0.095


In [49]:
magRange = np.linspace(15,30,16)

In [50]:
magRange

array([ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.])

In [57]:
catName = 'test0.cat'

assoc = Table.read(catName,format='ascii.sextractor')
t0 = time.time()
df = assoc.to_pandas()
#df2 = df[df['MAG_BEST'] != 99.0000]
print(time.time()-t0) #, totGross)


0.00117707252502


In [56]:
t0 = time.time()
df_spark = sqlContext.createDataFrame(df)
print(time.time()-t0) #, totGross)

t0 = time.time()
totGross = df_spark.count()
print(time.time()-t0, totGross)

1.73592019081
(0.06544804573059082, 16094)


In [59]:
!head test0.cat

#   1 NUMBER                 Running object number                                     
#   2 MAG_BEST               Best of MAG_AUTO and MAG_ISOCOR                            [mag]
#   3 X_IMAGE                Object position along x                                    [pixel]
#   4 Y_IMAGE                Object position along y                                    [pixel]
#   5 ELLIPTICITY            1 - B_IMAGE/A_IMAGE                                       
         1  20.7173    744.5268     19.6496    0.438
         2  99.0000   1000.7009     19.5017    0.285
         3  99.0000    133.3164     20.2157    0.325
         4  21.8619    138.0000     20.4564    0.499
         5  21.5303    779.9139     23.0000    0.633


In [27]:
x = 0
vals = [[x],[1],[2]]
val = [[x],[3]]
print(val)
x = 1
val = [[x],[3]]
print(val)
vals.append(val[0])

[[0], [3]]
[[1], [3]]


In [4]:
import numpy as np 
smallName = 'test'

In [5]:
valListminus1 = [1,3,5,7,9,11]
for vals in (valListminus1):
    valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[vals]]

    finalTableTuples = []
    #p = ThreadPool(1)
    #t0 = time.time()
    for x0 in range(3): #len(valList[0])):
        for x1 in range(3): #len(valList[1])):
                for x2 in range(3): #len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    print(fullValList)
#                     try:
#                         finalTableTuples.extend(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))

test_0_0_0_0
[[1.0, 1.0, 8.0, 1]]
test_0_0_1_0
[[1.0, 1.0, 16.0, 1]]
test_0_0_2_0
[[1.0, 1.0, 24.0, 1]]
test_0_1_0_0
[[1.0, 2.0, 8.0, 1]]
test_0_1_1_0
[[1.0, 2.0, 16.0, 1]]
test_0_1_2_0
[[1.0, 2.0, 24.0, 1]]
test_0_2_0_0
[[1.0, 3.0, 8.0, 1]]
test_0_2_1_0
[[1.0, 3.0, 16.0, 1]]
test_0_2_2_0
[[1.0, 3.0, 24.0, 1]]
test_1_0_0_0
[[2.0, 1.0, 8.0, 1]]
test_1_0_1_0
[[2.0, 1.0, 16.0, 1]]
test_1_0_2_0
[[2.0, 1.0, 24.0, 1]]
test_1_1_0_0
[[2.0, 2.0, 8.0, 1]]
test_1_1_1_0
[[2.0, 2.0, 16.0, 1]]
test_1_1_2_0
[[2.0, 2.0, 24.0, 1]]
test_1_2_0_0
[[2.0, 3.0, 8.0, 1]]
test_1_2_1_0
[[2.0, 3.0, 16.0, 1]]
test_1_2_2_0
[[2.0, 3.0, 24.0, 1]]
test_2_0_0_0
[[3.0, 1.0, 8.0, 1]]
test_2_0_1_0
[[3.0, 1.0, 16.0, 1]]
test_2_0_2_0
[[3.0, 1.0, 24.0, 1]]
test_2_1_0_0
[[3.0, 2.0, 8.0, 1]]
test_2_1_1_0
[[3.0, 2.0, 16.0, 1]]
test_2_1_2_0
[[3.0, 2.0, 24.0, 1]]
test_2_2_0_0
[[3.0, 3.0, 8.0, 1]]
test_2_2_1_0
[[3.0, 3.0, 16.0, 1]]
test_2_2_2_0
[[3.0, 3.0, 24.0, 1]]
test_0_0_0_0
[[1.0, 1.0, 8.0, 3]]
test_0_0_1_0
[[1.0, 1.0, 16.0,